<a href="https://colab.research.google.com/github/jcmachicaocuf/codigos_CUF_LLM_NLP/blob/main/U4__LLM__herramientas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Fine-tuning vs. RAG (Búsqueda con recuperación de contexto)


In [ ]:
from transformers import pipeline
import wikipedia

# RAG-style retrieval
def retrieve_context(query):
    results = wikipedia.search(query)
    if results:
        summary = wikipedia.summary(results[0], sentences=2)
        return summary
    return "No context found."

# Generación con contexto
qa_pipeline = pipeline("text-generation", model="facebook/opt-125m")

query = "¿Quién fue Nikola Tesla?"
context = retrieve_context(query)
response = qa_pipeline(f"Contexto: {context} Pregunta: {query} Respuesta:")

print(response[0]['generated_text'])


# 2. Cuantización para reducir tamaño de modelos

In [ ]:
from transformers import AutoModelForCausalLM
import torch

# Carga del modelo en precisión reducida
model_id = "facebook/opt-125m"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

print(f"Modelo {model_id} cargado con cuantización en float16")

# 3. Simulación de RLHF (Entrenamiento con retroalimentación humana)

In [ ]:
import numpy as np

# Simulación de feedback humano en RLHF
responses = ["Buena respuesta", "Respuesta irrelevante", "Respuesta útil", "Incorrecto"]
rewards = {"Buena respuesta": 1.0, "Respuesta útil": 0.5, "Incorrecto": -1.0, "Respuesta irrelevante": -0.5}

# Simulación de entrenamiento
def simulate_feedback():
    response = np.random.choice(responses)
    return response, rewards[response]

for _ in range(5):
    response, reward = simulate_feedback()
    print(f"Modelo generó: {response} | Recompensa: {reward}")

# 4. IA Simbólica con Reglas y LLM

In [ ]:
def symbolic_reasoning(question):
    rules = {
        "Es un mamífero y pone huevos": "ornitorrinco",
        "Tiene plumas y vuela": "ave",
        "Tiene escamas y vive en agua": "pez"
    }

    for rule, answer in rules.items():
        if rule in question:
            return f"Según el razonamiento simbólico, la respuesta es: {answer}"

    return "No se encontró una respuesta basada en reglas."

print(symbolic_reasoning("Es un mamífero y pone huevos"))

# 5. Interacción entre agentes LLM

In [ ]:
class Agent:
    def __init__(self, name):
        self.name = name

    def respond(self, message):
        return f"{self.name} responde: '{message[::-1]}'"  # Simulación de respuesta

agent1 = Agent("Agente A")
agent2 = Agent("Agente B")

message = "Hola, cómo estás?"
print(agent1.respond(message))
print(agent2.respond(agent1.respond(message)))

# 6. Uso de LangChain para generación de reportes estructurados

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI  # O usar un modelo local como GPT4All

# Definir plantilla de informe estructurado
template = """
Eres un asistente especializado en generar reportes técnicos.
Responde con información clara y estructurada.

Tema: {topic}

[Resumen]
Escribe un resumen del tema en 2-3 líneas.

[Datos clave]
Enumera 3 puntos clave sobre el tema.

[Conclusión]
Da una conclusión concisa y bien fundamentada.
"""

prompt = PromptTemplate(input_variables=["topic"], template=template)
llm = OpenAI(model_name="gpt-3.5-turbo")  # Cambiar por otro LLM si es necesario
chain = LLMChain(llm=llm, prompt=prompt)

# Solicitar informe sobre un tema
response = chain.run(topic="Impacto de la IA en la educación")
print(response)

# 7. LangGraph para decisión basada en reglas con LLM


In [ ]:
from langgraph.graph import StateGraph, END
from langchain.llms import OpenAI
from langchain.schema import Document

# Definir estados
class State:
    def __init__(self, query):
        self.query = query
        self.result = None

# Nodo 1: Buscar en una base de datos antes de generar texto
def search_database(state):
    knowledge_base = {
        "IA en salud": "Los modelos de IA se usan en diagnóstico y personalización de tratamientos.",
        "Automatización en fábricas": "Los robots industriales han aumentado la eficiencia en la manufactura."
    }
    if state.query in knowledge_base:
        state.result = knowledge_base[state.query]
        return END  # Termina aquí si hay respuesta en la base de datos
    return "llm_generation"  # Pasa al siguiente paso

# Nodo 2: Generar texto si no hay información previa
def llm_generation(state):
    llm = OpenAI(model_name="gpt-3.5-turbo")
    state.result = llm.predict(f"Explica brevemente: {state.query}")
    return END

# Construcción del grafo
graph = StateGraph(State)
graph.add_node("search_database", search_database)
graph.add_node("llm_generation", llm_generation)
graph.set_entry_point("search_database")
graph.add_edge("search_database", "llm_generation")

# Ejecutar flujo
state = State("IA en salud")
graph.run(state)
print(state.result)

 # 8. LangChain para estructurar contratos en JSON modulares

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-4")  # Usar modelo adecuado

# Nivel 1: Análisis de estructura contractual
structure_prompt = PromptTemplate(
    input_variables=["contract_text"],
    template="Extrae la estructura del contrato en un JSON con las cláusulas y sus relaciones clave."
)
structure_chain = LLMChain(llm=llm, prompt=structure_prompt)

# Nivel 2: Análisis de riesgos
risk_prompt = PromptTemplate(
    input_variables=["contract_structure"],
    template="Analiza los riesgos en base a la estructura dada y devuelve un JSON con los principales riesgos."
)
risk_chain = LLMChain(llm=llm, prompt=risk_prompt)

# Ejemplo de entrada (contrato ficticio)
contract_text = "El arrendador se obliga a entregar el inmueble en buen estado. El arrendatario debe pagar mensualmente."

# Generación de JSON de estructura
json_structure = structure_chain.run(contract_text)
print("Estructura contractual:", json_structure)

# Generación de JSON de riesgos (basado en estructura previa)
json_risks = risk_chain.run(json_structure)
print("Análisis de riesgos:", json_risks)

# 9. Código: Gestión Modular de Proyectos con JSON

In [ ]:
import json

# Definir células JSON independientes
task_json = {
    "tasks": [
        {"id": 1, "name": "Diseño UI", "dependency": None, "duration": 10},
        {"id": 2, "name": "Backend API", "dependency": 1, "duration": 15},
        {"id": 3, "name": "Pruebas", "dependency": 2, "duration": 7}
    ]
}

risk_json = {
    "risks": [
        {"task_id": 2, "risk": "Retraso en integración con frontend", "probability": 0.3},
        {"task_id": 3, "risk": "Errores críticos en pruebas", "probability": 0.5}
    ]
}

resource_json = {
    "resources": [
        {"id": 1, "name": "Diseñador UX", "assigned_to": 1},
        {"id": 2, "name": "Desarrollador Backend", "assigned_to": 2},
        {"id": 3, "name": "QA Tester", "assigned_to": 3}
    ]
}

# Función para articular información de diferentes células
def generate_project_view(view_type):
    if view_type == "timeline":
        return json.dumps(task_json, indent=4)

    elif view_type == "risk_analysis":
        return json.dumps(risk_json, indent=4)

    elif view_type == "resource_allocation":
        return json.dumps(resource_json, indent=4)

    elif view_type == "full_project":
        project_data = {**task_json, **risk_json, **resource_json}
        return json.dumps(project_data, indent=4)

    else:
        return "Vista no reconocida."

# Ejemplo de uso: Generar vista de riesgos
print("🔍 Análisis de Riesgos:")
print(generate_project_view("risk_analysis"))

# Ejemplo de uso: Generar vista completa del proyecto
print("\n📊 Vista Completa del Proyecto:")
print(generate_project_view("full_project"))

# 10. Ejemplo de Código con Dos Ontologías de Calidad

In [ ]:
import json

# Definimos dos ontologías de calidad
quality_ontologies = {
    "ISO9001": {
        "focus": "Procesos Estandarizados",
        "priorities": ["Documentación", "Estandarización", "Auditoría"],
        "workflow_adjustments": {"add_review_phase": True, "reduce_iterations": False}
    },
    "AgileQuality": {
        "focus": "Flexibilidad e Iteración",
        "priorities": ["Colaboración", "Iteraciones Rápidas", "Feedback"],
        "workflow_adjustments": {"add_review_phase": False, "reduce_iterations": True}
    }
}

# Plan inicial sin ajustes de calidad
project_plan = {
    "tasks": [
        {"id": 1, "name": "Diseño", "duration": 10},
        {"id": 2, "name": "Desarrollo", "duration": 15},
        {"id": 3, "name": "Pruebas", "duration": 7}
    ]
}

def apply_quality_ontology(ontology_name):
    """Ajusta el plan según la ontología de calidad seleccionada."""
    ontology = quality_ontologies.get(ontology_name)
    if not ontology:
        return "Ontología no reconocida."

    modified_plan = project_plan.copy()

    if ontology["workflow_adjustments"]["add_review_phase"]:
        modified_plan["tasks"].append({"id": 4, "name": "Revisión Final", "duration": 5})

    if ontology["workflow_adjustments"]["reduce_iterations"]:
        for task in modified_plan["tasks"]:
            task["duration"] = max(1, int(task["duration"] * 0.8))  # Reducimos duración en 20%

    return json.dumps(modified_plan, indent=4)

# Comparación entre dos estrategias
print("🔍 Estrategia ISO9001:")
print(apply_quality_ontology("ISO9001"))

print("\n🚀 Estrategia AgileQuality:")
print(apply_quality_ontology("AgileQuality"))